In [1]:
import tensorflow as tf
print("TF version:",tf.__version__)
#判断TF是否支持GPU
print("GPU is","available" if tf.test.is_gpu_available() else "NOT AVAILABLE")


TF version: 2.1.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available


In [2]:
import tensorflow as tf
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 不显示等级2以下的提示信息

print('GPU', tf.test.is_gpu_available())

a = tf.constant(2.0)
b = tf.constant(4.0)
print(a + b)

GPU True
tf.Tensor(6.0, shape=(), dtype=float32)


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
import glob
from IPython import display

In [4]:
#导入FashionMNIST数据集
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images,train_labels),(test_image,test_labels)=fashion_mnist.load_data()

In [5]:
#数据预处理，
train_images = train_images.reshape(train_images.shape[0],28,28,1).astype('float32')
train_images = (train_images-127.5)/127.5  #Normalization  pictures，使其数据处于[-1,1]之间

In [6]:
buffer_size = 60000
batch_size = 256

In [7]:
#批量化打乱数据
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(buffer_size).batch(batch_size)

In [8]:
#构建生成器
def make_generator_model():
    #全连接层
    model=tf.keras.Sequential()#keras的序列模型
    model.add(layers.Dense(7*7*256,use_bias=False,input_shape=(1000,)))
    model.add(layers.BatchNormalization())#标准化
    model.add(layers.LeakyReLU())#激活，传统ReLU的改进激活函数
    #上采样层          
    model.add(layers.Reshape((7,7,256)))
    assert model.output_shape ==(None,7,7,256) 
    
    model.add(layers.Conv2DTranspose(128,(5,5),strides=(1,1),padding='same',use_bias=False))#反卷积
    assert model.output_shape ==(None,7,7,128)#生成结果
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2DTranspose(64,(5,5),strides=(2,2),padding='same',use_bias=False))#反卷积
    assert model.output_shape == (None,14,14,64)#生成结果
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2DTranspose(1,(5,5),strides=(2,2),padding='same',use_bias=False,activation='tanh'))
    assert model.output_shape == (None,28,28,1)
    
    return model


In [9]:
#查看生成器网络结构
generator = make_generator_model()
generator.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12544)             12544000  
_________________________________________________________________
batch_normalization (BatchNo (None, 12544)             50176     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 12544)             0         
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 256)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 7, 7, 128)         819200    
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 7, 128)         512       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 7, 7, 128)         0

In [10]:
#使用未训练的生成器训练一张图片
#noise = tf.random.normal([1,100])
#generated_image = generator(noise,training=False)

#plt.imshow(generated_image[0,:,:,0],cmap='gray')


In [11]:
#构建判别器 可以视为一个CNN分类器
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64,(5,5),strides=(2,2),padding = 'same',input_shape =[28,28,1]))
    
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(128,(5,5),strides=(2,2),padding="same"))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    
    return model

In [12]:
#查看判别网络结构
discriminator = make_discriminator_model()
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 64)        1664      
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 128)         204928    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 7, 7, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)             

In [13]:
#查看判别器的判别结果
#decision = discriminator(generated_image)
#print(decison)

In [14]:
#定义损失函数和优化器
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [15]:
#判别器损失函数
#判断和0之间的差异
def discriminator_loss(real_output,fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output),real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output),fake_output)
    total_losee = real_loss +fake_loss
    return total_loss

#生成器损失函数
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output),fake_output)

#两者的优化器
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)


In [16]:
#定义检查点
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir,"ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer = discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [17]:
#模型训练
epochs = 100
noise_dim =100
num_examples_to_generate = 16

In [18]:
#产生随机种子作为输入
seed = tf.random.normal([num_examples_to_generate,noise_dim])


In [19]:
#定义单次训练过程
#加注告诉编译器可以静态图方式执行
#训练在生成器接收到随机种子作为输入的开始没用于生成一张图片，判别器随后区分真实图片（选自训练集）还是伪造图片（生成器生成）
#针对每一个模型都计算损失函数，并计算梯度，用于更新生成器和判别器
@tf.function
def train_step(images):
    noise = tf.random.normal([batch_size,noise_dim])
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise,training=True)
        
        real_output = discriminator(images,training=True)
        fake_output = discriminator(generated_images,training=True)
        
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output,fake_output)
        
    gradients_of_generator = gen_tape.gradient(gen_loss,generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss,discriminator.trainable_variables)
    
    generator_optimizer.apply_gradients(zip(gradients_of_generator,trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator,discriminator.trainable_variables))

In [20]:
#生成和保存图片
def generate_and_save_images(model,epoch,test_input):
#所有层在推理模式下运行（batchnorm）
    predictions = model(test_input,training=False)
    
    fig = plt.figure(figsize=(4,4))
    
    for i in range(predictions.shape[0]):
        plt.subplot(4,4,i+1)
        plt.imshow(predictions[i,:,:,0]*127.5+127.5,cmap='gray')
        plt.axis('off')
        
    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

In [21]:
#定义循环训练过程
def train(dataset,epochs):
    for epoch in range(epochs):
        start = time.time()
        
        for image_batch in dataset:
            train_step(image_batch)
            
        #生成图片
        display.clear_output(wait=True)
        generate_and_save_images(generator,epoch + 1,seed)
        
        #每5 epoch 进行一次存储
        if (epoch +1)%5 ==0:
            checkpoint.save(file_prefix = checkpoint_prefix)
        print('Time for epoch {} is {}'.format(epoch+1,time.time()-start))
        

In [22]:
%%time
train(train_dataset,epochs)


ValueError: in converted code:

    <ipython-input-19-7df1c15f5f9d>:10 train_step  *
        generated_images = generator(noise,training=True)
    D:\Anaconda\envs\tf2\lib\site-packages\tensorflow_core\python\keras\engine\base_layer.py:737 __call__
        self.name)
    D:\Anaconda\envs\tf2\lib\site-packages\tensorflow_core\python\keras\engine\input_spec.py:213 assert_input_compatibility
        ' but received input with shape ' + str(shape))

    ValueError: Input 0 of layer sequential is incompatible with the layer: expected axis -1 of input shape to have value 1000 but received input with shape [256, 100]


In [ ]:
#展示某一个epoch
def display_image(epoch_no):
    return PIL.Image.open('image_at_epoch{:04d}.png'.format(epoch_no))

In [ ]:
display_image(epochs)

In [ ]:
#合成训练过程产生的gif图
import imageio #需要另外安装
anim_file ='dcgan.gif'

with imageio.get_writer(anim_file,mode='I') as writer:
    filenames =glob.glob('image*.png')
    filenames = sorted(filenames)
    last= -1
    for i,filename in enumerate(filenames):
        frame = 2*(i**0.5)
        if round(fram,e)> round(last):
            last = frame
        else:
            continue
        image = imageio.imread(filemane)
        write.append_data(image)
        image = imageio.imread(filename)
        write.append_data(image)
        #生成的GIF在文件夹中
        